In [1]:
import pandas as pd
import openpyxl

In [2]:
# Read xlsx file and conver to dataframe
df = pd.read_excel('Emisiones.xlsx', sheet_name='Report')

In [3]:
# Print resumen of the dataframe
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158234 entries, 0 to 158233
Data columns (total 24 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   131550 non-null  object 
 1   Unnamed: 1   0 non-null       float64
 2   Unnamed: 2   8866 non-null    object 
 3   Unnamed: 3   128502 non-null  object 
 4   Unnamed: 4   76 non-null      object 
 5   Unnamed: 5   0 non-null       float64
 6   Unnamed: 6   0 non-null       float64
 7   Unnamed: 7   114 non-null     object 
 8   Unnamed: 8   158 non-null     float64
 9   Unnamed: 9   128582 non-null  object 
 10  Unnamed: 10  128620 non-null  object 
 11  Unnamed: 11  128462 non-null  object 
 12  Unnamed: 12  0 non-null       float64
 13  Unnamed: 13  128462 non-null  object 
 14  Unnamed: 14  196 non-null     object 
 15  Unnamed: 15  0 non-null       float64
 16  Unnamed: 16  128658 non-null  object 
 17  Unnamed: 17  0 non-null       float64
 18  Unnamed: 18  128658 non-

In [4]:
# print first elements of the dataframe
print(df.head(25))

                        Unnamed: 0  Unnamed: 1  \
0                              NaN         NaN   
1                              NaN         NaN   
2                              NaN         NaN   
3                              NaN         NaN   
4                           Póliza         NaN   
5                              NaN         NaN   
6                              NaN         NaN   
7   A  0301       215    13    105         NaN   
8   A  0301       215    13    106         NaN   
9   A  0301       215    13    108         NaN   
10  A  0301       215    13    109         NaN   
11                             NaN         NaN   
12                             NaN         NaN   
13                    Reasegurador         NaN   
14                             NaN         NaN   
15                     MS AMLIN AG         NaN   
16                             NaN         NaN   
17      KOREAN REINSURANCE COMPANY         NaN   
18                             NaN         NaN   


In [5]:
# Initialize an empty list to store the selected DataFrames
selected_dfs = []

# Flags to track whether "TOTAL:" and "Contrato:" are found
total_found = False
contrato_found = False

productos_vida =['VIDA', 'VIDA COLECTIVO - CANCELACION DE DEUDAS']

# Iterate through the DataFrame
for i in range(len(df)):
    # Check conditions for each row
    if (
        df.iloc[i, 2] == "TOTAL:" and
        df.iloc[i, 3] == 2023 and
        (df.iloc[i, 7]).split()[0] == "VIDA" and
        df.iloc[i, 4] == "EXCEDENTE"
    ):
        total_found = True
        contrato_found = False
    elif df.iloc[i, 2] == "Contrato:":
        contrato_found = True
        total_found = False
    
    # Select rows between "TOTAL:" and "Contrato:"
    if total_found and not contrato_found:
        selected_dfs.append(df.iloc[i])

# Check if there are selected DataFrames
if selected_dfs:
    # Concatenate the list of selected DataFrames into a single DataFrame
    result_df = pd.concat(selected_dfs, axis=1).T
    
    # Filter out rows that start with specific words in the first column
    result_df = result_df[~result_df.iloc[:, 0].astype(str).str.startswith(('Póliza', 'Operador:'))]
    
    # Filter out rows that in the third column start with specific words
    result_df = result_df[~result_df.iloc[:, 2].astype(str).str.startswith(('REGIONAL', 'Listado', 'Emisiones'))]
    
    # Remove columns with all NaN values
    result_df = result_df.dropna(axis=1, how='all')
    
    # Now, result_df contains the desired rows between "TOTAL:" and "Contrato:" with additional filtering
else:
    print("No matching rows found.")

result_df.to_excel('output.xlsx', index=False)

In [6]:
result_df.to_excel('output.xlsx', index=False)

In [7]:
def filter_dataframe_vida(df, year, riesgo, vida=False):
    # Initialize an empty list to store the selected DataFrames
    selected_dfs = []

    # Flags to track whether "TOTAL:" and "Contrato:" are found
    total_found = False
    contrato_found = False

    # Iterate through the DataFrame
    for i in range(len(df)):
        # Check conditions for each row
        if (
            df.iloc[i, 2] == "TOTAL:" and
            df.iloc[i, 3] == year and
            (df.iloc[i, 7]).split()[0] == "VIDA" and
            df.iloc[i, 4] == riesgo
        ):
            total_found = True
            contrato_found = False
        elif df.iloc[i, 2] == "Contrato:":
            contrato_found = True
            total_found = False
    
        # Select rows between "TOTAL:" and "Contrato:"
        if total_found and not contrato_found:
            selected_dfs.append(df.iloc[i])

    # Check if there are selected DataFrames
    if selected_dfs:
        # Concatenate the list of selected DataFrames into a single DataFrame
        result_df = pd.concat(selected_dfs, axis=1).T

        # Filter out rows that start with specific words in the first column
        result_df = result_df[~result_df.iloc[:, 0].astype(str).str.startswith(('Póliza', 'Operador:'))]

        # Filter out rows that in the third column start with specific words
        result_df = result_df[~result_df.iloc[:, 2].astype(str).str.startswith(('REGIONAL', 'Listado', 'Emisiones'))]

        # Remove columns with all NaN values
        result_df = result_df.dropna(axis=1, how='all')

        # Change the column names for letters. Example: "Unamed: 0" to "A"
        result_df.columns = [chr(65 + i) for i in range(len(result_df.columns))]

        return result_df
    else:
        print("No matching rows found.")
        return None
"""
# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
# Replace 2022 and 'EXCEDENTE' with the desired values for year and riesgo
your_filtered_df = filter_dataframe(df, 2023, 'EXCEDENTE')
if your_filtered_df is not None:
    print(your_filtered_df)
"""

"\n# Example usage\n# Replace 'your_dataframe' with the actual DataFrame variable name\n# Replace 2022 and 'EXCEDENTE' with the desired values for year and riesgo\nyour_filtered_df = filter_dataframe(df, 2023, 'EXCEDENTE')\nif your_filtered_df is not None:\n    print(your_filtered_df)\n"

In [8]:
def filter_dataframe_patrimoniales(df, year, riesgo, vida=False):
    # Initialize an empty list to store the selected DataFrames
    selected_dfs = []

    # Flags to track whether "TOTAL:" and "Contrato:" are found
    total_found = False
    contrato_found = False

    # Iterate through the DataFrame
    for i in range(len(df)):
        # Check conditions for each row
        if (
            df.iloc[i, 2] == "TOTAL:" and
            df.iloc[i, 3] == year and
            not (df.iloc[i, 7]).split()[0] == "VIDA" and
            df.iloc[i, 4] == riesgo
        ):
            total_found = True
            contrato_found = False
        elif df.iloc[i, 2] == "Contrato:":
            contrato_found = True
            total_found = False
    
        # Select rows between "TOTAL:" and "Contrato:"
        if total_found and not contrato_found:
            selected_dfs.append(df.iloc[i])

    # Check if there are selected DataFrames
    if selected_dfs:
        # Concatenate the list of selected DataFrames into a single DataFrame
        result_df = pd.concat(selected_dfs, axis=1).T

        # Filter out rows that start with specific words in the first column
        result_df = result_df[~result_df.iloc[:, 0].astype(str).str.startswith(('Póliza', 'Operador:'))]

        # Filter out rows that in the third column start with specific words
        result_df = result_df[~result_df.iloc[:, 2].astype(str).str.startswith(('REGIONAL', 'Listado', 'Emisiones'))]

        # Remove columns with all NaN values
        result_df = result_df.dropna(axis=1, how='all')

        # Change the column names for letters. Example: "Unamed: 0" to "A"
        result_df.columns = [chr(65 + i) for i in range(len(result_df.columns))]

        return result_df
    else:
        print("No matching rows found.")
        return None
"""
# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
# Replace 2022 and 'EXCEDENTE' with the desired values for year and riesgo
your_filtered_df = filter_dataframe(df, 2023, 'EXCEDENTE')
if your_filtered_df is not None:
    print(your_filtered_df)
"""

"\n# Example usage\n# Replace 'your_dataframe' with the actual DataFrame variable name\n# Replace 2022 and 'EXCEDENTE' with the desired values for year and riesgo\nyour_filtered_df = filter_dataframe(df, 2023, 'EXCEDENTE')\nif your_filtered_df is not None:\n    print(your_filtered_df)\n"

In [9]:

# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
# Replace 2022 and 'EXCEDENTE' with the desired values for year and riesgo
filtered_vida = filter_dataframe_vida(df, 2023, 'EXCEDENTE', vida=False)
filtered_patrimoniales = filter_dataframe_patrimoniales(df, 2023, 'EXCEDENTE', vida=False)
filtered_vida.to_excel('output_vida.xlsx', index=False)
filtered_patrimoniales.to_excel('output_patrimoniales.xlsx', index=False)


In [10]:

def remove_totals(df):
    # Remove all the rows that start with "TOTAL:" in the second column
    result_df = df[~df.iloc[:, 1].astype(str).str.startswith('TOTAL:')]
    
    # Remove all empty rows
    result_df = result_df.dropna(how='all')

    return result_df
"""
# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
your_processed_df = process_dataframe(your_dataframe)
print(your_processed_df)
"""

"\n# Example usage\n# Replace 'your_dataframe' with the actual DataFrame variable name\nyour_processed_df = process_dataframe(your_dataframe)\nprint(your_processed_df)\n"

In [11]:
def calculate_sums(df):
    # Suma los valores numéricos de las columnas H, la columna L y la columna M,
    # devuélveme los valores y almacénalos en variables "prima", "importe_comision" y "a_favor" respectivamente
    prima = pd.to_numeric(df['H'], errors='coerce').fillna(0).sum()
    importe_comision = pd.to_numeric(df['L'], errors='coerce').fillna(0).sum()
    a_favor = pd.to_numeric(df['M'], errors='coerce').fillna(0).sum()

    return prima, importe_comision, a_favor

In [12]:
def process_and_sum(df, year, riesgo):
    # Step 1: Filter the DataFrame
    filtered_df = filter_dataframe(df, year, riesgo)
    
    # Step 2: Remove totals from the filtered DataFrame
    processed_df = remove_totals(filtered_df)
    
    # Step 3: Calculate the sums
    sums_result = calculate_sums(processed_df)
    
    return sums_result

# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
# Replace 2022 and 'EXCEDENTE' with the desired values for year and riesgo
your_sums_result = process_and_sum(df, 2023, 'EXCEDENTE')
print("Prima:", your_sums_result[0])
print("Importe Comision:", your_sums_result[1])
print("A Favor:", your_sums_result[2])

NameError: name 'filter_dataframe' is not defined

In [ ]:
print(result_df.head())

In [ ]:
# Change the column names for letters. Example: "Unamed: 0" to "A"
result_df.columns = [chr(65 + i) for i in range(len(result_df.columns))]

In [ ]:
# Remove all the rows that start with "TOTAL:" in the second column, and remove all empty rows
result_df = result_df[~result_df.iloc[:, 1].astype(str).str.startswith('TOTAL:')].dropna(how='all')

In [ ]:
# Suma los valores numéricos de las columnas H, la columna L y la columna M,
# devuélveme los valores y almacénalos en variables "prima", "importe_comision" y "a_favor" respectivamente
prima = pd.to_numeric(result_df['H'], errors='coerce').fillna(0).sum()
importe_comision = pd.to_numeric(result_df['L'], errors='coerce').fillna(0).sum()
a_favor = pd.to_numeric(result_df['M'], errors='coerce').fillna(0).sum()

# Imprime los valores de las variables "prima", "importe_comision" y "a_favor"
print(prima)
print(importe_comision)
print(a_favor)

In [ ]:
def filter_recuperos_patrimoniales(df, year, riesgo, vida=False):
    # Initialize an empty list to store the selected DataFrames
    selected_dfs = []

    # Flags to track whether "TOTAL:" and "Contrato:" are found
    total_found = False
    contrato_found = False

    # Iterate through the DataFrame
    for i in range(len(df)):
        # Check conditions for each row
        # Qué secciones calcular: en este caso se excluyen VIDA y CAUCION
        if (
            df.iloc[i, 0] == "Total Contrato:" and
            df.iloc[i, 2] == year and
            not (df.iloc[i, 9]).split()[0] == "VIDA" and
            not (df.iloc[i, 9]).split()[0] == "CAUCION" and
            df.iloc[i, 3] == riesgo
        ):
            total_found = True
            contrato_found = False
        elif df.iloc[i, 0] == "Contrato:" or df.iloc[i, 1] == "Resumen":
            contrato_found = True
            total_found = False
    
        # Select rows between "TOTAL:" and "Contrato:"
        if total_found and not contrato_found:
            selected_dfs.append(df.iloc[i])

    # Check if there are selected DataFrames
    if selected_dfs:
        # Concatenate the list of selected DataFrames into a single DataFrame
        result_df = pd.concat(selected_dfs, axis=1).T

        # Filter out rows that start with specific words in the first column
        result_df = result_df[~result_df.iloc[:, 0].astype(str).str.startswith(('Póliza', 'Operador:'))]

        # Filter out rows that in the third column start with specific words
        result_df = result_df[~result_df.iloc[:, 2].astype(str).str.startswith(('REGIONAL', 'Listado', 'Emisiones', 'Anulaciones'))]

        # Remove columns with all NaN values
        result_df = result_df.dropna(axis=1, how='all')

        # Change the column names for letters. Example: "Unamed: 0" to "A"
        result_df.columns = [chr(65 + i) for i in range(len(result_df.columns))]

        return result_df
    else:
        print("No matching rows found.")
        return None
"""
# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
# Replace 2022 and 'EXCEDENTE' with the desired values for year and riesgo
your_filtered_df = filter_recuperos_patrimoniales(df, 2023, 'EXCEDENTE')
if your_filtered_df is not None:
    print(your_filtered_df)
"""

"\n# Example usage\n# Replace 'your_dataframe' with the actual DataFrame variable name\n# Replace 2022 and 'EXCEDENTE' with the desired values for year and riesgo\nyour_filtered_df = filter_dataframe_patrimoniales(df, 2023, 'EXCEDENTE')\nif your_filtered_df is not None:\n    print(your_filtered_df)\n"

In [ ]:
def remove_totals_recuperos(df):
    # Remove all the rows that start with "TOTAL:" in the second column
    result_df = df[~df.iloc[:, 0].astype(str).str.startswith('Total Contrato:')]
    
    # Remove all empty rows
    result_df = result_df.dropna(how='all')

    return result_df
"""
# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
your_processed_df = remove_totals_recuperos(your_dataframe)
print(your_processed_df)
"""

"\n# Example usage\n# Replace 'your_dataframe' with the actual DataFrame variable name\nyour_processed_df = remove_totals_recuperos(your_dataframe)\nprint(your_processed_df)\n"

In [ ]:
def calculate_sums_recuperos(df):
    # Suma los valores numéricos de las columnas H, la columna L y la columna M,
    # devuélveme los valores y almacénalos en variables "prima", "importe_comision" y "a_favor" respectivamente
    indemnizacion = pd.to_numeric(df['J'], errors='coerce').fillna(0).sum()
    gastos = pd.to_numeric(df['M'], errors='coerce').fillna(0).sum()
    importe_total = pd.to_numeric(df['O'], errors='coerce').fillna(0).sum()

    return indemnizacion, gastos, importe_total

In [ ]:
recuperos_df = filter_recuperos_patrimoniales(df, 2023, 'EXCEDENTE', vida=False)

In [ ]:
no_totals_df = remove_totals_recuperos(recuperos_df)

In [ ]:
no_totals_df.to_excel('output_recuperos_no_totals.xlsx', index=False)

In [ ]:
sums_result = calculate_sums_recuperos(no_totals_df)
print("Indemnizacion:", sums_result[0])
print("Gastos:", sums_result[1])
print("Importe Total:", sums_result[2])

Indemnizacion: 638095112.0
Gastos: 16133964.0
Importe Total: 654229076.0


In [ ]:
recuperos_df.to_excel('output_recuperos.xlsx', index=False)